# Pipeline ELT

## Extração e Carregamento


Em ELT/cargaRaw.py, os dados são extraídos dos arquivos .CSV da prefeitura que estão na pasta data/

Utilizando a bibilioteca Pandas, extraímos os dados de 2003 até 2005, e de 2018 até 2020. O dados são enviados para o banco de dados um por um. Os dados anteriores e posteriores a 2016 são segregados em duas tabelas diferentes pois valores monetários a partir de 2016 são formatados com vírgula como separador decimal, enquanto os anteriores são formatados com ponto. 


In [3]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()

DATABASE_URL = f"postgresql+psycopg2://{os.getenv('DATABASE_USER')}:{os.getenv('DATABASE_PASSWORD')}@{os.getenv('DATABASE_HOST')}:{os.getenv('DATABASE_PORT')}/{os.getenv('DATABASE_NAME')}"
engine = create_engine(DATABASE_URL)

with engine.begin() as conn:
    RESULT = conn.execute(text("SELECT NOW();")) 
    for row in RESULT:
        print(f"Connexão feita com sucesso: {row[0]}")

Connexão feita com sucesso: 2025-08-08 21:02:05.207225-03:00


In [4]:
import pandas as pd

# Lista dos anos que serão processados na abordagem ELT
anos = (2003, 2004, 2005, 2018, 2019, 2020)

print("=== INICIANDO CARGA ELT - DADOS BRUTOS ===")
print("Carregando dados sem transformação na tabela raw_despesas")

for ano in anos:
    print(f"Processando ano {ano}...")
    caminho = f"../data/recife-dados-despesas-{ano}.csv"
    
    # Extração: lê os dados CSV sem nenhuma transformação
    df = pd.read_csv(caminho, sep=';', encoding='utf-8')    
    
    # Load: insere dados brutos diretamente no banco para posterior transformação
    # if_exists="append" adiciona os dados sem sobrescrever registros existentes
    if ano < 2016:
        df.to_sql("raw_despesas_pre_2016", con=engine, if_exists="append", index=False)
    else:
        # Para anos >= 2016, cria uma tabela separada para evitar problemas com vírgula/ponto
        df.to_sql("raw_despesas_pos_2016", con=engine, if_exists="append", index=False)
    
    print(f"  ✓ {len(df)} registros carregados para {ano}")

print("\n=== CARGA ELT CONCLUÍDA ===")
print("Dados brutos disponíveis nas tabelas 'raw_despesas_pre_2016' e 'raw_despesas_pos_2016' para transformação")


=== INICIANDO CARGA ELT - DADOS BRUTOS ===
Carregando dados sem transformação na tabela raw_despesas
Processando ano 2003...
  ✓ 101373 registros carregados para 2003
Processando ano 2004...
  ✓ 103990 registros carregados para 2004
Processando ano 2005...
  ✓ 95430 registros carregados para 2005
Processando ano 2018...
  ✓ 106164 registros carregados para 2018
Processando ano 2019...
  ✓ 117323 registros carregados para 2019
Processando ano 2020...
  ✓ 102699 registros carregados para 2020

=== CARGA ELT CONCLUÍDA ===
Dados brutos disponíveis nas tabelas 'raw_despesas_pre_2016' e 'raw_despesas_pos_2016' para transformação


## Transformação


Em ELT/transformacaoDados.py os dados são transformados já no banco de dados postgres, utilizando a biblioteca SQLAlchemy para executar comandos SQL diretamente no banco. Os dados passam por 4 etapas principais:

Primeiro, em cada uma das tabelas de dados `raw_despesas_pre_2016` e `raw_despesas_pos_2016`, encontramos todas colunas que são texto e removemos os espaços em branco no início e no fim de cada valor, tornamos todos os caracteres minúsculos e substituímos espaços em branco por underline.

In [5]:
def clean_text(conn, table):
    # Encontra todas as colunas que são texto
        result = conn.execute(text(f"""
        SELECT column_name 
        FROM information_schema.columns
        WHERE table_name = '{table}'
        AND data_type = 'text';
        """))
        
        columns = [row.column_name for row in result]
        
        # Trim espaços brancos, deixa tudo minúsculo, substitui ' ' por '_'
        for column in columns:
            print(f"Transformação iniciada para {table}.{column}")
            conn.execute(text(f"""
                UPDATE {table}
                SET {column} = REPLACE(LOWER(TRIM({column})), ' ', '_')
                WHERE {column} IS NOT NULL;
            """))
            print(f"Transformação de {table}.{column} finalizada")
            
with engine.begin() as conn:

    for table in ('raw_despesas_pre_2016', 'raw_despesas_pos_2016'):
        clean_text(conn, table)

Transformação iniciada para raw_despesas_pre_2016.orgao_nome
Transformação de raw_despesas_pre_2016.orgao_nome finalizada
Transformação iniciada para raw_despesas_pre_2016.unidade_nome
Transformação de raw_despesas_pre_2016.unidade_nome finalizada
Transformação iniciada para raw_despesas_pre_2016.categoria_economica_nome
Transformação de raw_despesas_pre_2016.categoria_economica_nome finalizada
Transformação iniciada para raw_despesas_pre_2016.grupo_despesa_nome
Transformação de raw_despesas_pre_2016.grupo_despesa_nome finalizada
Transformação iniciada para raw_despesas_pre_2016.modalidade_aplicacao_nome
Transformação de raw_despesas_pre_2016.modalidade_aplicacao_nome finalizada
Transformação iniciada para raw_despesas_pre_2016.elemento_nome
Transformação de raw_despesas_pre_2016.elemento_nome finalizada
Transformação iniciada para raw_despesas_pre_2016.subelemento_nome
Transformação de raw_despesas_pre_2016.subelemento_nome finalizada
Transformação iniciada para raw_despesas_pre_2016.

Após isso, procuramos por todas as colunas que representam valores monetários em ambas tabelas. Em `raw_despesas_pos_2016`, estas colunas são text, então trocamos ',' por '.', garantimos que não há caracteres não númericos e então convertemos para numeric. Em `raw_despesas_pre_2016`, estas colunas são double precision, então também as convertemos para numeric.

In [6]:
def valores_monetarios_para_numeric(conn):
    COL_NUMERIC = ('valor_empenhado', 'valor_liquidado', 'valor_pago')
    
    for column in COL_NUMERIC:
        print(f"Transformação iniciada para 'raw_despesas_pos_2016'.{column}")
        
        # Primeiro, limpa os dados removendo caracteres não numéricos (exceto ponto e vírgula)
        # e padroniza o separador decimal para ponto
        conn.execute(text(f"""
            UPDATE raw_despesas_pos_2016
            SET {column} = CASE
                WHEN {column} IS NULL OR TRIM(CAST({column} AS TEXT)) = '' THEN NULL
                ELSE CAST(
                    REPLACE(
                        REPLACE(
                            REGEXP_REPLACE(CAST({column} AS TEXT), '[^0-9,.\\-]', '', 'g'),
                            ',', '.'
                        ),
                        '..', '.'
                    ) AS NUMERIC(18,2)
                )
            END;
        """))
                    
        # Depois altera o tipo da coluna para NUMERIC(18,2)
        for table in ('raw_despesas_pre_2016', 'raw_despesas_pos_2016'):
            conn.execute(text(f"""
                ALTER TABLE {table}
                ALTER COLUMN {column} TYPE NUMERIC(18,2)
                USING {column}::NUMERIC(18,2);
            """))
        
            print(f"Transformação de {table}.{column} finalizada")

with engine.begin() as conn:
    valores_monetarios_para_numeric(conn)

Transformação iniciada para 'raw_despesas_pos_2016'.valor_empenhado
Transformação de raw_despesas_pre_2016.valor_empenhado finalizada
Transformação de raw_despesas_pos_2016.valor_empenhado finalizada
Transformação iniciada para 'raw_despesas_pos_2016'.valor_liquidado
Transformação de raw_despesas_pre_2016.valor_liquidado finalizada
Transformação de raw_despesas_pos_2016.valor_liquidado finalizada
Transformação iniciada para 'raw_despesas_pos_2016'.valor_pago
Transformação de raw_despesas_pre_2016.valor_pago finalizada
Transformação de raw_despesas_pos_2016.valor_pago finalizada


Em seguida, convertemos a coluna unidade_codigo para string, pois ela foi inicialmente carregada como double precision por conter um ponto mas na verdade é um código de unidade que não deve ser tratado como número. 

In [7]:
def tratar_unidade_codigo(conn, table):
    conn.execute(text(f"""
            ALTER TABLE {table}
            ALTER COLUMN unidade_codigo TYPE text
            USING unidade_codigo::text;
    """))
    
with engine.begin() as conn:

    for table in ('raw_despesas_pre_2016', 'raw_despesas_pos_2016'):
        tratar_unidade_codigo(conn, table)

Por fim, criamos a tabela `despesas_recife` caso ela já não tenha sido criado pelo processo de ETL, e inserimos os dados transformados de ambas as tabelas `raw_despesas_pre_2016` e `raw_despesas_pos_2016` nela.

In [9]:
def unificar_tabelas_despesas(conn):
    # Cria a tabela unificada com base na estrutura de uma das tabelas
    print("Criando tabela unificada 'despesas_recife'...")
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS despesas_recife AS
        SELECT * FROM raw_despesas_pre_2016 WHERE 1=0;
    """))
    
    # Insere dados da primeira tabela
    print("Inserindo dados de 'raw_despesas_pre_2016'...")
    conn.execute(text("""
        INSERT INTO despesas_recife
        SELECT * FROM raw_despesas_pre_2016;
    """))
    
    # Insere dados da segunda tabela
    print("Inserindo dados de 'raw_despesas_pos_2016'...")
    conn.execute(text("""
        INSERT INTO despesas_recife
        SELECT * FROM raw_despesas_pos_2016;
    """))

with engine.begin() as conn:
    unificar_tabelas_despesas(conn)

Criando tabela unificada 'despesas_recife'...
Inserindo dados de 'raw_despesas_pre_2016'...
Inserindo dados de 'raw_despesas_pos_2016'...


In [11]:
query_df = pd.read_sql("SELECT * FROM despesas_recife TABLESAMPLE SYSTEM (1) WHERE ano_movimentacao BETWEEN 2003 AND 2005 OR ano_movimentacao BETWEEN 2018 AND 2020 ORDER BY VALOR_PAGO DESC LIMIT 10;", engine)
query_df

,ano_movimentacao,mes_movimentacao,orgao_codigo,orgao_nome,unidade_codigo,unidade_nome,categoria_economica_codigo,categoria_economica_nome,grupo_despesa_codigo,grupo_despesa_nome,...,empenho_numero,subempenho,indicador_subempenho,credor_codigo,credor_nome,modalidade_licitacao_codigo,modalidade_licitacao_nome,valor_empenhado,valor_liquidado,valor_pago
0,2018,12,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,881,92,x,0,credor_não_informado,0,não_informada,23957636.69,23957636.69,23957636.69
1,2018,7,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,881,87,x,0,credor_não_informado,0,não_informada,21650209.34,21650209.34,21650209.34
2,2018,3,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,881,83,x,0,credor_não_informado,0,não_informada,21582066.05,21582066.05,21582066.05
3,2018,6,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,881,86,x,0,credor_não_informado,0,não_informada,21478019.94,21478019.94,21478019.94
4,2018,9,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,881,89,x,0,credor_não_informado,0,não_informada,21409262.88,21409262.88,21409262.88
5,2018,11,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,881,91,x,0,credor_não_informado,0,não_informada,19271429.25,19271429.25,19271429.25
6,2020,1,50,secretaria_de_infraestrutura_-_administração_s...,50.1,autarquia_de_manutenção_e_limpeza_urbana_-_emlurb,3,despesas_correntes,3,outras_despesas_correntes,...,3,6,s,6000141,vital_engenharia_ambiental_s/a,89,concorrencia,0.00,5086147.61,5086147.61
7,2020,4,14,secretaria_de_educação,14.01,secretaria_de_educação_-_administração_direta,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,118,84,x,0,credor_não_informado,0,não_informada,3753908.72,3753908.72,3753908.72
8,2020,10,80,encargos_gerais_do_muncípio,80.01,recursos_sob_a_gestão_da_secretaria_de_finanças,3,despesas_correntes,3,outras_despesas_correntes,...,686,0,n,4500130,ministerio_da_fazenda_-_procuradoria_da_fazend...,98,dispensado,3235864.82,3235864.82,3235864.82
9,2018,4,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,3,outras_despesas_correntes,...,293,5,s,3800015,irmandade_da_santa_casa_de_misericordia_do_rec...,93,convenio,0.00,2723622.31,2723622.31
